In [1]:
x_train_file_path = 'train_x.csv/train_x.csv'
x_test_file_path = 'test_x.csv/test_x.csv'
y_train_file_path = 'train_y.csv'
sample_sub = 'sample_submission.csv'

In [24]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from keras.layers import Dense, SimpleRNN
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
# get the corrisponding labels
y_train = pd.read_csv(y_train_file_path, low_memory=False)
x_train = pd.read_csv(x_train_file_path, low_memory=False)
x_test = pd.read_csv(x_test_file_path, low_memory=False)
sample_set = pd.read_csv(sample_sub)

In [4]:
x_train = x_train.dropna(axis=1, how='all')
x_train = x_train.drop('cellattributevalue', axis=1)
x_train = x_train.drop('celllabel', axis=1)
x_train = x_train.drop('nursingchartcelltypevalname', axis=1)
x_train = x_train.drop('nursingchartvalue', axis=1)
x_train = x_train.drop('labmeasurenamesystem', axis=1)
x_train = x_train.drop('labname', axis=1)
x_train = x_train.drop('labresult', axis=1)
x_train = x_train.dropna()

In [5]:
x_test = x_test.dropna(axis=1, how='all')
x_test = x_test.drop('cellattributevalue', axis=1)
x_test = x_test.drop('celllabel', axis=1)
x_test = x_test.drop('nursingchartcelltypevalname', axis=1)
x_test = x_test.drop('nursingchartvalue', axis=1)
x_test = x_test.drop('labmeasurenamesystem', axis=1)
x_test = x_test.drop('labname', axis=1)
x_test = x_test.drop('labresult', axis=1)
x_test = x_test.drop('Unnamed: 0', axis=1)
x_test = x_test.dropna()

In [6]:
merged_df = pd.merge(x_train, y_train, on='patientunitstayid', how='inner')
merged_df = merged_df.drop('Unnamed: 0_x', axis=1)
merged_df = merged_df.drop('Unnamed: 0_y', axis=1)
merged_df['age'] = pd.to_numeric(merged_df['age'], errors='coerce')
merged_df.dropna(inplace=True)

In [7]:
df1 = pd.get_dummies(merged_df['ethnicity'])
merged_df = pd.concat([merged_df, df1], axis=1).reindex(merged_df.index)
merged_df.drop('ethnicity', axis=1, inplace=True)
df2 = pd.get_dummies(merged_df['gender'])
merged_df = pd.concat([merged_df, df2], axis=1).reindex(merged_df.index)
merged_df.drop('gender', axis=1, inplace=True)

In [8]:
y_train = merged_df.pop('hospitaldischargestatus')

In [9]:
# loop through each column in the DataFrame
for col in merged_df.columns:
    # check if the column data type is float64
    if merged_df[col].dtype == 'float64':
        # convert the column to float32
        merged_df[col] = merged_df[col].astype('float32')

In [10]:
x_test['age'] = pd.to_numeric(x_test['age'], errors='coerce')
x_test.dropna(inplace=True)

    
# endoced dummies
df4 = pd.get_dummies(x_test['ethnicity'])
x_test = pd.concat([x_test, df4], axis=1).reindex(x_test.index)
x_test.drop('ethnicity', axis=1, inplace=True)

df3 = pd.get_dummies(x_test['gender'])
x_test = pd.concat([x_test, df3], axis=1).reindex(x_test.index)
x_test.drop('gender', axis=1, inplace=True)


# fix merged_df
merged_df = merged_df.drop('Native American', axis=1)

In [11]:
x_test

,admissionheight,admissionweight,age,offset,patientunitstayid,unitvisitnumber,African American,Asian,Caucasian,Hispanic,Other/Unknown,Female,Male
1,165.1,86.80,66.0,0.0,151900.0,1.0,0,0,1,0,0,1,0
2,170.2,81.00,41.0,0.0,152954.0,1.0,0,0,1,0,0,1,0
3,193.0,163.80,39.0,0.0,158056.0,1.0,0,0,1,0,0,0,1
4,162.6,49.20,20.0,0.0,159411.0,1.0,0,0,1,0,0,1,0
6,180.3,104.30,83.0,0.0,162502.0,1.0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,185.4,140.60,47.0,0.0,3343295.0,1.0,1,0,0,0,0,0,1
500,165.1,129.80,63.0,0.0,3344944.0,2.0,0,0,1,0,0,1,0
501,172.7,75.00,73.0,0.0,3347502.0,1.0,0,0,1,0,0,0,1
502,160.0,49.44,87.0,0.0,3348293.0,2.0,0,0,1,0,0,1,0


In [21]:
# Create SVM classifier
classifier = svm.SVC(kernel='sigmoid', C=0.1, gamma='auto')

# Train the model
classifier.fit(merged_df, y_train)

# Make predictions
y_pred = classifier.predict(merged_df)

# Evaluate the model
accuracy = classifier.score(merged_df, y_train)
print('Accuracy:', accuracy)

Accuracy: 0.925476603119584


In [22]:
y_pred = classifier.predict(x_test)

In [23]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [26]:
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(merged_df.shape[1], 1)))
model.add(Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(merged_df, y_train, epochs=10, batch_size=32)

# Use model to predict labels of new data
y_pred = model.predict(x_test)

# Convert predicted labels back to dataframe format
df_pred = pd.DataFrame(y_pred, columns=['predicted_label'])

Epoch 1/10
55/55 [==============================] - 2s 4ms/step - loss: 0.3496 - accuracy: 0.8758
Epoch 2/10
55/55 [==============================] - 0s 3ms/step - loss: 0.2658 - accuracy: 0.9255
Epoch 3/10
55/55 [==============================] - 0s 3ms/step - loss: 0.2656 - accuracy: 0.9255
Epoch 4/10
55/55 [==============================] - 0s 3ms/step - loss: 0.2658 - accuracy: 0.9255
Epoch 5/10
55/55 [==============================] - 0s 4ms/step - loss: 0.2652 - accuracy: 0.9255
Epoch 6/10
55/55 [==============================] - 0s 3ms/step - loss: 0.2654 - accuracy: 0.9255
Epoch 7/10
55/55 [==============================] - 0s 3ms/step - loss: 0.2648 - accuracy: 0.9255
Epoch 8/10
55/55 [==============================] - 0s 4ms/step - loss: 0.2662 - accuracy: 0.9255
Epoch 9/10
55/55 [==============================] - 0s 4ms/step - loss: 0.2649 - accuracy: 0.9255
Epoch 10/10
14/14 [==============================] - 0s 2ms/step


In [25]:
merged_df.shape[1]

13

In [38]:
# Extract one column from the first dataframe
col = x_test['patientunitstayid']
#col
#df_pred
# Concatenate the column with the second dataframe
result = pd.concat([col, df_pred], axis=1)
result = result.dropna()
result

,patientunitstayid,predicted_label
1,151900.0,0.069404
2,152954.0,0.073927
3,158056.0,0.069404
4,159411.0,0.073927
6,162502.0,0.073927
...,...,...
438,3063333.0,0.056100
439,3069831.0,0.071239
440,3075364.0,0.073927
441,3077438.0,0.071239


In [29]:
sample_set

,patientunitstayid,hospitaldischargestatus
0,151179,0.090282
1,151900,0.292194
2,152954,0.038120
3,158056,0.258662
4,159411,0.765303
...,...,...
499,3343295,0.988567
500,3344944,0.924342
501,3347502,0.539642
502,3348293,0.065770
